In [3]:
!pip install numpy==1.23.5
!pip install pathlib
!pip install ruamel.yaml
!pip install daal==2021.2.3
!pip install --prefer-binary pyarrow
!pip install transformers datasets torch

     |████████████████████████████████| 195.4 MB 149 kB/s eta 0:00:011   |████████▌                       | 52.0 MB 929 kB/s eta 0:02:35
     |████████████████████████████████| 640 kB 28.7 MB/s eta 0:00:01
  Attempting uninstall: tbb
    Found existing installation: TBB 0.2
ERROR: Cannot uninstall 'TBB'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.


In [9]:
import os 
import pandas as pd
import numpy as np
from datetime import datetime

In [10]:
# Get the current directory
current_directory = os.getcwd()

print(f"current directory: {current_directory}")

current directory: /Users/hayleyluo/Desktop/Github/ML-kaggle-on-the-snow-ski-area-reviews/kaggle-ski-area-review


In [11]:
# import the dataset
file = f"{current_directory}/datasets/bronze/on_the_snow_ski_area_reviews.csv"
df = pd.read_csv(file, header=0)
df

,Unnamed: 0,State,Ski Area,Reviewer Name,Review Date,Review Star Rating (out of 5),Review Text,Review Missing,Review Date datetime
0,0.0,california,squaw-valley-usa,philip sayles,31st December 2019,3.0,I'm glad our family experienced Squaw but I wo...,1.0,2019-12-31
1,1.0,california,squaw-valley-usa,mateonelson,22nd May 2019,4.0,"I went skiing today 5/22, granite chief was am...",1.0,2019-05-22
2,2.0,california,squaw-valley-usa,anonymous_user,7th March 2019,2.0,We had a horrible experience on our family ski...,1.0,2019-03-07
3,3.0,california,squaw-valley-usa,Ivan Cazares,5th March 2019,3.0,This is the first year I ski Squaw. I've been ...,1.0,2019-03-05
4,4.0,california,squaw-valley-usa,welzbob,26th February 2019,3.0,"Both Squaw and Alpine have incredible terrain,...",1.0,2019-02-26
...,...,...,...,...,...,...,...,...,...
18381,18257.0,new-hampshire,whaleback-mountain,Matthijs,18th March 2017,1.0,Looks like they only made snow on one trail. Y...,1.0,2017-03-18
18382,18258.0,new-hampshire,whaleback-mountain,Dolly,25th February 2017,3.0,Looks cute from highway........ but not enough...,1.0,2017-02-25
18383,18259.0,new-hampshire,whaleback-mountain,Roger,11th February 2017,3.0,Place has incredible potential. Could be best ...,1.0,2017-02-11
18384,18260.0,new-hampshire,whaleback-mountain,Arthur,10th February 2017,2.0,Great exposure from interstate......... but lo...,1.0,2017-02-10


In [12]:
from transformers import pipeline

# Load sentiment analysis pipeline
sentiment_analyzer = pipeline("sentiment-analysis")

# Quick test
sentiment_analyzer("The ski resort was absolutely amazing and the snow was perfect!")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use mps:0


[{'label': 'POSITIVE', 'score': 0.9998791217803955}]

In [16]:
import pandas as pd
from tqdm import tqdm

# Optional: Clean or sample your data
reviews = df["Review Text"].dropna().tolist()

# Apply Hugging Face sentiment analysis
results = []
for review in tqdm(reviews):
    result = sentiment_analyzer(review[:512])[0]  # Truncate to 512 tokens
    results.append(result)

# Add results to the DataFrame
df = df.loc[df["Review Text"].notnull()].copy()
df["label"] = [r["label"] for r in results]
df["score"] = [r["score"] for r in results]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 18250/18250 [1:34:48<00:00,  3.21it/s]


In [31]:
def check_output(true_label, label):
    if true_label>3.0 and label == "POSITIVE":
        return "CORRECT"
    elif true_label<3.0 and label == "NEGATIVE":
        return "CORRECT"
    else:
        return "AMBIGUOUS WITH THE TRUE RATING"

In [32]:
# df["cleaned_review"] = df["review_text"].apply(clean_text)

df["verdict"] = df.apply(lambda row: check_output(row["Review Star Rating (out of 5)"], row["label"]), axis=1)
df

,Unnamed: 0,State,Ski Area,Reviewer Name,Review Date,Review Star Rating (out of 5),Review Text,Review Missing,Review Date datetime,label,score,verdict
0,0.0,california,squaw-valley-usa,philip sayles,31st December 2019,3.0,I'm glad our family experienced Squaw but I wo...,1.0,2019-12-31,NEGATIVE,0.999641,AMBIGUOUS WITH THE TRUE RATING
1,1.0,california,squaw-valley-usa,mateonelson,22nd May 2019,4.0,"I went skiing today 5/22, granite chief was am...",1.0,2019-05-22,POSITIVE,0.998424,CORRECT
2,2.0,california,squaw-valley-usa,anonymous_user,7th March 2019,2.0,We had a horrible experience on our family ski...,1.0,2019-03-07,NEGATIVE,0.992195,CORRECT
3,3.0,california,squaw-valley-usa,Ivan Cazares,5th March 2019,3.0,This is the first year I ski Squaw. I've been ...,1.0,2019-03-05,NEGATIVE,0.997998,AMBIGUOUS WITH THE TRUE RATING
4,4.0,california,squaw-valley-usa,welzbob,26th February 2019,3.0,"Both Squaw and Alpine have incredible terrain,...",1.0,2019-02-26,POSITIVE,0.994726,AMBIGUOUS WITH THE TRUE RATING
...,...,...,...,...,...,...,...,...,...,...,...,...
18381,18257.0,new-hampshire,whaleback-mountain,Matthijs,18th March 2017,1.0,Looks like they only made snow on one trail. Y...,1.0,2017-03-18,NEGATIVE,0.998799,CORRECT
18382,18258.0,new-hampshire,whaleback-mountain,Dolly,25th February 2017,3.0,Looks cute from highway........ but not enough...,1.0,2017-02-25,NEGATIVE,0.996284,AMBIGUOUS WITH THE TRUE RATING
18383,18259.0,new-hampshire,whaleback-mountain,Roger,11th February 2017,3.0,Place has incredible potential. Could be best ...,1.0,2017-02-11,NEGATIVE,0.995321,AMBIGUOUS WITH THE TRUE RATING
18384,18260.0,new-hampshire,whaleback-mountain,Arthur,10th February 2017,2.0,Great exposure from interstate......... but lo...,1.0,2017-02-10,NEGATIVE,0.997501,CORRECT


In [33]:
df[df['verdict']=="AMBIGUOUS WITH THE TRUE RATING"]

,Unnamed: 0,State,Ski Area,Reviewer Name,Review Date,Review Star Rating (out of 5),Review Text,Review Missing,Review Date datetime,label,score,verdict
0,0.0,california,squaw-valley-usa,philip sayles,31st December 2019,3.0,I'm glad our family experienced Squaw but I wo...,1.0,2019-12-31,NEGATIVE,0.999641,AMBIGUOUS WITH THE TRUE RATING
3,3.0,california,squaw-valley-usa,Ivan Cazares,5th March 2019,3.0,This is the first year I ski Squaw. I've been ...,1.0,2019-03-05,NEGATIVE,0.997998,AMBIGUOUS WITH THE TRUE RATING
4,4.0,california,squaw-valley-usa,welzbob,26th February 2019,3.0,"Both Squaw and Alpine have incredible terrain,...",1.0,2019-02-26,POSITIVE,0.994726,AMBIGUOUS WITH THE TRUE RATING
9,9.0,california,squaw-valley-usa,anonymous_user,2nd March 2018,4.0,"I skied Squaw Valley today in the morning, 3/1...",1.0,2018-03-02,NEGATIVE,0.998953,AMBIGUOUS WITH THE TRUE RATING
14,14.0,california,squaw-valley-usa,D Sullivan,12th March 2017,3.0,Booked 3 days of skiing But only skied 1 1/2 d...,1.0,2017-03-12,NEGATIVE,0.996558,AMBIGUOUS WITH THE TRUE RATING
...,...,...,...,...,...,...,...,...,...,...,...,...
18378,18254.0,new-hampshire,whaleback-mountain,David Cook,4th February 2017,2.0,My dad and I visited on a whim on our way back...,1.0,2017-02-04,POSITIVE,0.998670,AMBIGUOUS WITH THE TRUE RATING
18380,18256.0,new-hampshire,whaleback-mountain,woodia,23rd January 2013,3.0,Lousiest snowboard experience in New England b...,1.0,2013-01-23,NEGATIVE,0.999523,AMBIGUOUS WITH THE TRUE RATING
18382,18258.0,new-hampshire,whaleback-mountain,Dolly,25th February 2017,3.0,Looks cute from highway........ but not enough...,1.0,2017-02-25,NEGATIVE,0.996284,AMBIGUOUS WITH THE TRUE RATING
18383,18259.0,new-hampshire,whaleback-mountain,Roger,11th February 2017,3.0,Place has incredible potential. Could be best ...,1.0,2017-02-11,NEGATIVE,0.995321,AMBIGUOUS WITH THE TRUE RATING


In [35]:
df.to_csv("results/hugging_face_results.csv")